# Stock Market Indices - Ticker Web Scraping and Data download

*** 
## Load required libraries

In [1]:
import  pylab as pl

from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np

from scipy.cluster.vq import kmeans,vq
from math import sqrt

from sklearn.cluster import KMeans

import pandas as pd
import pandas_datareader as dr

from matplotlib import pyplot as plt

***
## Scrape ticker data
There is no consistency among wiki pages that list index constituents. You will need to adjust your code to make sure you can download most of the available data. 

### 1. Execute **one** of the four market cells below:

In [2]:
# US market - S&P500
index_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies' 
ticker_column_name='Symbol'  # This is the name of the column containing tickers on the Wikipedia page. 
yahoo_suffix=''
table_idx=0

In [3]:
# Brazilian stock exchange - Ibovespa
index_url = 'https://en.wikipedia.org/wiki/List_of_companies_listed_on_B3' 
ticker_column_name='Ticker[4]'
yahoo_suffix='.SA'
table_idx=0

In [13]:
# UK - FTSE
index_url = 'https://en.wikipedia.org/wiki/FTSE_250_Index'
ticker_column_name='Ticker[4]' #'Ticker'
yahoo_suffix='.L'
table_idx=2 #0

In [5]:
# Canada - TSX
#index_url = 'https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index'
#ticker_column_name='Ticker'
#yahoo_suffix='.TO'
#table_idx=1  # for example, in this link, the list of companies is located in the second table

### 2. Scrape ticker data from the link

In [12]:
data_table

[                  0                                           1
 0                                                              
 1        Foundation                                        1984
 2          Operator                                  FTSE Group
 3         Exchanges                       London Stock Exchange
 4      Constituents              250 (consisting of 101 to 350)
 5              Type                                     Mid cap
 6        Market cap  £419.0 billion(as at 30 September 2021)[1]
 7  Weighting method               Capitalisation-weighted index
 8   Related indices                FTSE 100 IndexFTSE 350 Index
 9           Website                        FTSE UK Index Series,
    Year Total return
 0  2012        26.1%
 1  2013        32.3%
 2  2014         3.7%
 3  2015        11.2%
 4  2016         6.7%
 5  2017        17.8%
 6  2018       -13.3%
 7  2019        28.9%
 8  2020        -4.6%
 9  2021        16.9%,
                                Company T

In [14]:
# read in the url and scrape ticker data
data_table = pd.read_html(index_url,keep_default_na=False)
# Option keep_default_na=False prevents pandas from reading “NA” as NaN
# This came up when reading in TSX data for ticker "NA" for National Bank of Canada. The ticker was read as nan value

# get the list of ticker symbols
tickers = data_table[table_idx][ticker_column_name].tolist()

In [15]:
data_table

[                  0                                           1
 0                                                              
 1        Foundation                                        1984
 2          Operator                                  FTSE Group
 3         Exchanges                       London Stock Exchange
 4      Constituents              250 (consisting of 101 to 350)
 5              Type                                     Mid cap
 6        Market cap  £419.0 billion(as at 30 September 2021)[1]
 7  Weighting method               Capitalisation-weighted index
 8   Related indices                FTSE 100 IndexFTSE 350 Index
 9           Website                        FTSE UK Index Series,
    Year Total return
 0  2012        26.1%
 1  2013        32.3%
 2  2014         3.7%
 3  2015        11.2%
 4  2016         6.7%
 5  2017        17.8%
 6  2018       -13.3%
 7  2019        28.9%
 8  2020        -4.6%
 9  2021        16.9%,
                                Company T

In [8]:
tickers[0:20]

['ARE',
 'AEM',
 'AC',
 'AGI',
 'AQN',
 'ATD',
 'AP.UN',
 'ALA',
 'AIF',
 'ARX',
 'ATZ',
 'AX.UN',
 'ACO.X',
 'ATA',
 'AUP',
 'ACB',
 'BTO',
 'BDGI',
 'BLDP',
 'BMO']

### 3. Amend tickers to be consistent with the Yahoo server

In [9]:
# Replace substring in list of strings 
# TCL.A - Transcontinental Inc. A-class shares is TCL-A.TO
adjusted_tickers = list(map(lambda st: str.replace(st, ".", "-"), tickers)) 

# For Yahoo tickers, some countries are listed with a specific suffix, e.g., Brasil's Sao Paolo has .SA
suffixed_tickers = [sub + yahoo_suffix for sub in adjusted_tickers]

suffixed_tickers[0:20]

['ARE.TO',
 'AEM.TO',
 'AC.TO',
 'AGI.TO',
 'AQN.TO',
 'ATD.TO',
 'AP-UN.TO',
 'ALA.TO',
 'AIF.TO',
 'ARX.TO',
 'ATZ.TO',
 'AX-UN.TO',
 'ACO-X.TO',
 'ATA.TO',
 'AUP.TO',
 'ACB.TO',
 'BTO.TO',
 'BDGI.TO',
 'BLDP.TO',
 'BMO.TO']

***

## Download data based on selected tickers

In [10]:
# First, try to download a small sample of stocks - for example the first 10 on the list
# Note, some indices are large and it may take 30-45 min for daily price data to download.

prices_list = []
for ticker in suffixed_tickers[0:10]:
    try:
        prices = dr.DataReader(ticker,'yahoo','01/01/2017')['Adj Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        prices_list.append(prices)
    except:
        pass
    prices_df = pd.concat(prices_list,axis=1)

prices_df.sort_index(inplace=True)

prices_df.head()

,ARE.TO,AEM.TO,AC.TO,AGI.TO,AQN.TO,ATD.TO,AP-UN.TO,ALA.TO,AIF.TO,ARX.TO
Date,,,,,,,,,,
2017-01-03,12.680073,53.505264,13.83,9.397234,8.958282,30.279463,29.815746,24.910864,27.946449,17.877571
2017-01-04,12.705331,53.255459,13.72,9.609924,8.982213,30.545923,29.452240,25.465569,28.664921,18.002655
2017-01-05,12.595877,55.790539,13.74,10.992440,8.950302,30.817225,29.468767,25.436378,28.222788,17.815042
2017-01-06,12.486419,54.948597,13.56,10.131994,8.942326,30.633129,29.443974,25.399883,28.001717,17.564896
2017-01-09,12.267507,53.940109,13.50,10.431702,8.934350,30.812382,29.270493,25.224712,27.863548,17.260029


In [11]:
# If the small set download went smoothly, download prices for all tickers:

prices_list = []
for ticker in suffixed_tickers:
    try:
        prices = dr.DataReader(ticker,'yahoo','01/01/2017')['Adj Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        prices_list.append(prices)
    except:
        pass
    prices_df = pd.concat(prices_list,axis=1)

prices_df.sort_index(inplace=True)

prices_df.head()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects